In [3]:
!pip install dfcx-scrapi

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.1/263.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 131.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=55d674c9682308bdb34ba8aab4a3842dd24ff2931618fcad28575ffd67c238c3
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [4]:
import sys

# Colab Auth needed to call Client Endpoints (i.e. vertexai)
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth as google_auth
    google_auth.authenticate_user()

In [5]:
PROJECT_ID = "qwiklabs-gcp-02-efbb3ae9da81"

In [13]:
from dfcx_scrapi.core.agents import Agents

a = Agents()
agent = a.create_agent(
    project_id=PROJECT_ID,
    display_name="SCRAPI Agents",
    gcp_region="us-central1",
    playbook_agent=True
)

panel = "(playbooks/00000000-0000-0000-0000-000000000000/basics//right-panel:simulator)"
print(f"AGENT LINK: https://vertexaiconversation.cloud.google.com/{agent.name}/{panel}")
print(f"AGENT NAME: {agent.name}")

AlreadyExists: 409 com.google.apps.framework.request.AlreadyExistsException: Agent with display name 'SCRAPI Agents' already exists in the project. Code: ALREADY_EXISTS

In [14]:
import requests
from google.oauth2 import id_token
from google.auth.transport.requests import Request
import google.auth
import json

project_id = "qwiklabs-gcp-02-efbb3ae9da81"
location_id = "us-central1"

#Replace your Agent Name
agent_name = "projects/qwiklabs-gcp-02-efbb3ae9da81/locations/us-central1/agents/10cc3600-b0ef-49b0-b459-d6c7bda80bc8"
endpoint_base = "https://us-central1-dialogflow.googleapis.com"

def get_access_token():
    """Gets an access token using google-auth library."""
    creds, project = google.auth.default()
    auth_req = Request()
    creds.refresh(auth_req)
    return creds.token

def update_generative_settings(new_model=None, new_language_code=None):
    """Updates the generative settings for the Conversational agent."""
    access_token = get_access_token()
    headers = {
        "Authorization": f"Bearer {access_token}",
        "Content-Type": "application/json",
    }
    generative_settings_name = f"{agent_name}/generativeSettings"
    url = f"{endpoint_base}/v3/{generative_settings_name}"
    payload = {}
    update_mask_fields = []

    if new_model is not None:
        payload["llmModelSettings"] = {"model": new_model}
        update_mask_fields.append("llmModelSettings.model")

    if new_language_code is not None:
        payload["languageCode"] = new_language_code
        update_mask_fields.append("languageCode")

    if not payload:
        print("No settings to update.")
        return

    params = {"updateMask": ",".join(update_mask_fields)}

    try:
        response = requests.patch(url, headers=headers, params=params, json=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        updated_settings = response.json()
        print("Generative settings updated successfully:")
        print(json.dumps(updated_settings, indent=2))
        return updated_settings
    except requests.exceptions.RequestException as e:
        print(f"Error updating generative settings: {e}")
        if response is not None:
            print(f"Response status code: {response.status_code}")
            print(f"Response body: {response.text}")
        return None

# Example usage:
update_generative_settings(new_model="gemini-2.0-flash-001", new_language_code="en")

Generative settings updated successfully:
{
  "languageCode": "en",
  "name": "projects/qwiklabs-gcp-02-efbb3ae9da81/locations/us-central1/agents/10cc3600-b0ef-49b0-b459-d6c7bda80bc8/generativeSettings",
  "llmModelSettings": {
    "model": "gemini-2.0-flash-001"
  }
}


{'languageCode': 'en',
 'name': 'projects/qwiklabs-gcp-02-efbb3ae9da81/locations/us-central1/agents/10cc3600-b0ef-49b0-b459-d6c7bda80bc8/generativeSettings',
 'llmModelSettings': {'model': 'gemini-2.0-flash-001'}}

In [15]:
from dfcx_scrapi.core.sessions import Sessions
s = Sessions()
session_id = s.build_session_id(agent.name)
session_id

NameError: name 'agent' is not defined

In [16]:
res = s.detect_intent(agent.name, session_id, "hey, how are you?")
s.parse_result(res)

NameError: name 'agent' is not defined

In [ ]:
 res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
 s.parse_result(res)

In [ ]:
from dfcx_scrapi.core.playbooks import Playbooks

p = Playbooks(agent.name)

playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
playbooks_map

In [ ]:
playbook = p.get_playbook(playbooks_map["Default Generative Playbook"])
print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

In [ ]:
playbook = p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    goal="You are a friendly Tesla service center agent.\nYour job is to help users book appointments and answer any questions they have.",
    instructions=["Greet the user.", "Answer any questions the have to the best of your ability."]
    )

print(f"GOAL: {playbook.goal}")
print(f"INSTRUCTIONS: {playbook.instruction}")

In [ ]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what kind of models do you have?")
s.parse_result(res)

In [ ]:
%mkdir get_weather
%cd get_weather

In [ ]:
%%writefile requirements.txt
functions-framework==3.*
firebase-admin==6.2.*
firebase-functions
requests==2.*

In [ ]:
!wget https://raw.githubusercontent.com/GoogleCloudPlatform/dfcx-scrapi/main/data/get_weather_tool.py
!mv get_weather_tool.py main.py
!gcloud config set project $PROJECT_ID
!gcloud functions deploy get_weather_fun --region "us-central1" --runtime python311 --trigger-http --entry-point main --no-allow-unauthenticated --gen2

In [ ]:
get_weather_spec = """
openapi: 3.0.0
info:
  title: get_weather
  version: 1.0.0

servers:
  - url: YOUR_CLOUD_RUN_ENDPOINT

paths:
  /get_weather_grid:
    get:
      summary: Returns the current grid information for a city and state
      operationId: get_weather_grid
      parameters:
        - name: latitude
          in: query
          required: true
          schema:
            type: string
        - name: longitude
          in: query
          required: true
          schema:
            type: string
      responses:
        '200':
          description: OK
          content:
            application/json:
              schema:
                type: object
                properties:
                  data:
                    type: string
"""

In [ ]:
from dfcx_scrapi.core.tools import Tools

t = Tools()

# Tool 1 - get_weather
weather_tool = t.build_open_api_tool(
    "get_weather_fun",
    spec=get_weather_spec,
    description="Get the current weather for the provided city."
    )
tool1 = t.create_tool(agent.name, weather_tool)
print(str(tool1).lower())


In [ ]:
 from dfcx_scrapi.core.playbooks import Playbooks

 p = Playbooks(agent_id=agent.name)

 instructions = [
     "Use the ${TOOL:get_weather_fun} to get the current city/state weather grid information.",
     "- If the user only provides the city you can assume the state unless otherwise specified."]

 # New Playbook
 weather_agent = p.create_playbook(
     agent.name,
     display_name="Weather Agent",
     referenced_tools=[tool1.name],
     goal="You are a senior weather advisor at a network broadcast station. Your job is to predict the weather!",
     instructions=instructions
 )

 playbooks_map = p.get_playbooks_map(agent.name, reverse=True)
 playbooks_map

In [ ]:
p.update_playbook(
    playbooks_map["Default Generative Playbook"],
    instructions=["If the user needs help with the weather, call ${PLAYBOOK: Weather Agent}"]
)

In [ ]:
from dfcx_scrapi.core.sessions import Sessions

s = Sessions()

session_id = s.build_session_id(agent.name)
res = s.detect_intent(agent.name, session_id, "what is the weather like in Austin, Texas?")
s.parse_result(res)


In [ ]:
session_id = s.build_session_id(agent.name)

conversation = [
    "What colors does the model 3 come in?",
    "What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership."
    ]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1

In [ ]:
session_id = s.build_session_id(agent.name)

conversation = [
    "What colors does the model 3 come in?",
    "What's the weather like in Boston right now? I'm thinking about walking to the Tesla dealership."
    ]

i = 1
for utterance in conversation:
  print(f"\n--- TURN {i} --- \n")
  res = s.detect_intent(agent.name, session_id, utterance)
  s.parse_result(res)
  i+=1